In [2]:
# constant
RAW_DATA_PATH = "../data/raw/marketing_campaign.csv" 
Export_DATA_PATH = "../data/preprocessed/cleaned/marketing_campaign.csv" 

In [3]:
import pandas as pd
import numpy as np 
from scipy import stats


from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors


from sklearn.preprocessing import StandardScaler


# set the max columns to none
pd.set_option('display.max_columns', None)

In [11]:
data = pd.read_csv(RAW_DATA_PATH,sep="\t").drop(['ID'],axis=1)
data.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


### constant features

In [12]:
# Z_CostContact and Z_Revenue are constant
data = data.drop(['Z_CostContact', 'Z_Revenue'], axis = 1)
data.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1
1,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0
2,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0
3,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0
4,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0


### missing values

In [13]:
print('null percentage =',format(data['Income'].isna().sum() / len(data['Income']),'0.2f'),"%")
data = data.dropna(subset=['Income'])  # Drop rows where 'Income' is missing
print('null percentage =',format(data['Income'].isna().sum() / len(data['Income']),'0.2f'),"%")


null percentage = 0.01 %
null percentage = 0.00 %


### Detecting and handling outliers

Strategies:
 - Demographics features: drop them
 - other numeric features like MntFruits and Recency: in Feature engineering part.


- univarite

In [14]:
# Demographics features

# univarite
Demographics_cols = ['Year_Birth', 'Income']

Q1 = data[Demographics_cols].quantile(0.25)
Q3 = data[Demographics_cols].quantile(0.75)
IQR = Q3 - Q1

before = data.shape[0]
data = data[~((data[Demographics_cols] < (Q1 - 1.5 * IQR)) | (data[Demographics_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
after = data.shape[0]
dropped = before - after
print(f" len droped rows {before - after}, percentage = {format((dropped / before),'.3f')}%")


 len droped rows 11, percentage = 0.005%


## Data Types


In [15]:
# date
data.Dt_Customer = pd.to_datetime(data.Dt_Customer, format="%d-%m-%Y")

In [16]:
# Convert object columns to categorical
for col in data.select_dtypes(include='object').columns:
    data[col] = data[col].astype('category')

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2205 entries, 0 to 2239
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Year_Birth           2205 non-null   int64         
 1   Education            2205 non-null   category      
 2   Marital_Status       2205 non-null   category      
 3   Income               2205 non-null   float64       
 4   Kidhome              2205 non-null   int64         
 5   Teenhome             2205 non-null   int64         
 6   Dt_Customer          2205 non-null   datetime64[ns]
 7   Recency              2205 non-null   int64         
 8   MntWines             2205 non-null   int64         
 9   MntFruits            2205 non-null   int64         
 10  MntMeatProducts      2205 non-null   int64         
 11  MntFishProducts      2205 non-null   int64         
 12  MntSweetProducts     2205 non-null   int64         
 13  MntGoldProds         2205 non-null   i

In [17]:
# save the dataframe
data.to_parquet(Export_DATA_PATH, index=False)
